In [1]:
#Packages
import cobra
from cobra import Reaction, Metabolite, Model
from cobra.io import read_sbml_model

#Model input
model=read_sbml_model('iJN678.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
sql_e = Metabolite(
    'sql_e',
    #formula='C11H21N2O7PRS',
    name='Squalene extracellular',
    compartment='e')


# Add the squalene exportation with MsbA EC 7.5.2.6  
#ATP + H2O + lipid A-core oligosaccharide[side 1] = ADP + phosphate + lipid A-core oligosaccharide[side 2]
R_SQLt = cobra.Reaction(id="R_SQLt", name="Squalene transport", subsystem="Hopene Biosynthesis", lower_bound=0, upper_bound=1000)
R_SQLt.add_metabolites({model.metabolites.atp_c: -1, # ATP Cytosol 
                        model.metabolites.h2o_c: -1, # H2O Cytosol
                        model.metabolites.sql_c: -1, # Squalene Cytosol
                        sql_e: 1, # Squalene extarcelular
                        model.metabolites.adp_c: 1, # ADP cytosol
                        model.metabolites.pi_c: 1, # phosphate cytosol
                              })

# create exchange reaction for Squalene
EX_sql_e = cobra.Reaction(id="EX_sql_e", name="Squalene exchange", subsystem="Hopene Biosynthesis", lower_bound=0, upper_bound=1000)
EX_sql_e.add_metabolites({
    sql_e: -1
})
model.add_reactions([R_SQLt, EX_sql_e])

In [3]:
model.objective = "EX_sql_e"
model.optimize()

In [4]:
from cobra.io import write_sbml_model, validate_sbml_model
write_sbml_model(model, filename='iJN678mod.xml')
report = validate_sbml_model(filename='iJN678mod.xml')

print(report)

(<Model iJN678 at 0x7fd9c2a35130>, {'SBML_FATAL': [], 'SBML_ERROR': [], 'SBML_SCHEMA_ERROR': [], 'SBML_WARNING': [], 'COBRA_FATAL': [], 'COBRA_ERROR': [], 'COBRA_WARNING': [], 'COBRA_CHECK': []})
